In [6]:
from TradingAgent import PairsTradingAgent
import numpy as np
import pandas as pd
from utils import compute_spread, zscore_normalization
from copy import deepcopy as dc
from tqdm import tqdm

In [7]:
raw_data = pd.read_csv("../data/raw_data.csv", index_col=0)
raw_data.index = pd.to_datetime(raw_data.index)
raw_data.head()

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,40.689999,40.910000,26.337500,57.610001,42.930000,22.950001,101.830002,91.970001,54.439999,35.840000,...,124.980133,29.83,68.769997,35.700001,77.459999,36.080002,51.912292,98.844658,66.489998,47.270000
2016-01-05,40.549999,40.520000,25.677500,57.369999,42.919998,23.033333,102.360001,92.339996,54.040001,36.240002,...,125.839996,29.90,70.070000,36.060001,78.120003,36.070000,51.782890,100.902916,64.820000,48.009998
2016-01-06,40.730000,41.230000,25.174999,57.380001,42.560001,23.070000,102.160004,91.019997,51.740002,35.360001,...,125.839996,29.24,66.440002,36.439999,77.470001,35.619999,51.416248,101.339806,62.230000,48.020000
2016-01-07,39.000000,40.450001,24.112499,57.209999,41.540001,23.046667,99.160004,89.110001,50.419998,34.520000,...,114.949997,28.16,60.189999,36.580002,76.230003,34.700001,49.662113,99.009712,59.410000,46.560001
2016-01-08,38.590000,40.369999,24.240000,55.650002,40.669998,22.806667,98.199997,87.849998,49.980000,34.389999,...,116.620003,27.90,57.740002,36.180000,74.690002,34.369999,48.986340,98.592232,59.250000,45.880001


In [8]:
def grid_search(trading_agent: PairsTradingAgent, interval: float, 
                if_fixed_capital:bool, trading_capital=1.0, trading_capital_ratio=0.05):
    best_return = -np.inf
    best_entry_threshold = 0
    for entry_threshold in np.arange(0.8, 2.0, interval):   
        backtest_return_series = trading_agent.trading_simulation(entry=entry_threshold, exit=0.5, if_fixed_capital=if_fixed_capital,
                                                                  trading_capital=trading_capital, trading_capital_ratio=trading_capital_ratio)
        backtest_return = backtest_return_series.iloc[-1].values[0]
        if backtest_return > best_return:
            best_return = backtest_return
            best_entry_threshold = entry_threshold
    
    return best_entry_threshold, best_return

In [9]:
trading_pairs = pd.read_csv("../cluster_find_trading_pairs/trading_pairs.csv", index_col=0)
# trading_pairs.insert(loc=len(trading_pairs.columns), column="best_threshold", value=None)
# trading_pairs.insert(loc=len(trading_pairs.columns), column="best_return", value=None)
trading_pairs.head()

,leg1,leg2,coint_t,coint_pvalue,hedge_ratio,adf_pvalue,hurst_exponent,half_life,zero_cross
0,A,AVY,-3.992017,0.007372,1.245725,0.001497,0.397680,37.003035,117
1,A,DGX,-4.205360,0.003580,0.561827,0.000161,0.422547,36.591205,96
2,A,MS,-3.843708,0.011842,0.597435,0.002614,0.407522,50.032469,78
3,A,MTD,-3.814590,0.012961,10.115935,0.001257,0.418362,40.976883,105
4,A,NDAQ,-4.611753,0.000798,0.395134,0.000121,0.373605,29.043458,90


In [10]:
for ind in tqdm(trading_pairs.index):
    leg1 = trading_pairs.loc[ind, "leg1"]
    leg2 = trading_pairs.loc[ind, "leg2"]

    spread = compute_spread(raw_data, leg1, leg2)
    spread = zscore_normalization(spread)
    
    trading_agent = PairsTradingAgent(raw_data.loc[:, leg1], raw_data.loc[:, leg2], spread, initial_capital=1.0)
    best_entry_threshold, best_return = grid_search(trading_agent, interval=0.05, if_fixed_capital=True, trading_capital=1.0)
    
    trading_pairs.loc[ind, "best_threshold"] = best_entry_threshold
    trading_pairs.loc[ind, "best_return"] = best_return
    trading_pairs.to_csv("./trading_pairs_potential_fixed_capital.csv")
    # break

100%|██████████| 2069/2069 [4:05:27<00:00,  7.12s/it]  
